In [1]:
import os
import time
import pandas as pd
import numpy as np

import rasterio

import dask_gateway
import dask.array as da

from joblib import load

import raster_to_features as rm
import data_sampling_workflow.sample_rasters as sr

import planetary_computer as pc
import rioxarray as rioxr
import xarray as xr
import dask.dataframe as daf
import dask.array as da

import math

In [2]:
# initialize DASK cluster
cluster = dask_gateway.GatewayCluster()
cluster.scale(16)

client = cluster.get_client()
client

Connection method: Cluster object,Cluster type: dask_gateway.GatewayCluster
Dashboard: https://pccompute.westeurope.cloudapp.azure.com/compute/services/dask-gateway/clusters/prod.af9562aa7f34498ebe3908169d4d09b7/status,


In [2]:
def ndwi_band(raster):   # CHECK BAND #s THEYRE WRONG
    x = raster.sel(band=2).astype('int16') 
    y = raster.sel(band=4).astype('int16') 
    return (x-y) / (x+y)

def ndvi_band(raster):
    x = raster.sel(band=4).astype('int16') 
    y = raster.sel(band=1).astype('int16') 
    return (x-y) / (x+y)

In [3]:
#scene_ids = pd.read_csv(os.path.join(os.getcwd(),'temp','coastal_scenes_ids_2020.csv')).itemid.loc[19:26]
itemid = 'ca_m_3411933_nw_11_060_20200522'

# ---------------------------------------
# open NAIP scene

item = sr.get_item_from_id(itemid)    # locate raster
href = pc.sign(item.assets["image"].href)
#raster = rioxr.open_rasterio(href, chunks={"x": int(10580/4), "y":int(12510/4)})           # open raster as dask array (MAYBE FIGURE OUT BETTER CHUNK SIZES)
raster = rioxr.open_rasterio(href)
raster = raster.drop_vars(['spatial_ref','x','y'])
raster

<xarray.DataArray (band: 4, y: 12510, x: 10580)>
[529423200 values with dtype=uint8]
Coordinates:
  * band     (band) int64 1 2 3 4
Dimensions without coordinates: y, x
Attributes:
    _FillValue:    0.0
    scale_factor:  1.0
    add_offset:    0.0

In [12]:
x_n = raster.x.shape[0]
y_n = raster.y.shape[0]
indices = np.arange(1,x_n*y_n+1).reshape(x_n,y_n)
#indices

In [7]:
# apply this mask to pixels to keep those with ndwi below thresh
ndwi_thresh = 0.3  # keeps everything below this thresh
ndwi = ndwi_band(raster)
#ndwi

ndwi_mask = ndwi.where(ndwi>ndwi_thresh).isnull()  # using bool mask for memory usage (nan too big) raster's _FillValue is 0 too
#ndwi_mask

not_water = ndwi_mask * raster
not_water

<xarray.DataArray (y: 12510, x: 10580, band: 4)>
array([[[ 22,  54,  62, 149],
        [ 34,  59,  68, 152],
        [ 29,  57,  66, 162],
        ...,
        [ 42,  57,  79, 112],
        [ 38,  66,  73, 114],
        [ 40,  59,  76, 102]],

       [[ 31,  55,  70, 131],
        [ 31,  58,  67, 138],
        [ 35,  62,  67, 149],
        ...,
        [ 33,  54,  68, 124],
        [ 34,  65,  70, 118],
        [ 41,  63,  77, 100]],

       [[ 48,  70,  84,  98],
        [ 40,  62,  73, 119],
        [ 43,  68,  74, 137],
        ...,
...
        ...,
        [161, 152, 132, 151],
        [140, 135, 122, 135],
        [159, 150, 134, 148]],

       [[  0,   0,   0,   0],
        [  0,   0,   0,   0],
        [  0,   0,   0,   0],
        ...,
        [159, 149, 134, 154],
        [153, 148, 128, 144],
        [147, 137, 125, 131]],

       [[  0,   0,   0,   0],
        [  0,   0,   0,   0],
        [  0,   0,   0,   0],
        ...,
        [160, 152, 134, 158],
        [162, 154, 131, 150],
        [132, 136, 124, 123]]], dtype=uint8)
Coordinates:
  * band     (band) int64 1 2 3 4
Dimensions without coordinates: y, x

In [13]:
#is_water_indices = indices.T * ~ndwi_mask

In [6]:
# apply this mask to pixels to keep those with ndwi below thresh
ndvi_thresh = 0.05  # keeps everything below this thresh
ndvi = ndvi_band(not_water)
#ndvi
ndvi_mask = ndvi.where(ndvi<ndvi_thresh).isnull()
ndvi_mask

is_veg = ndvi_mask * not_water
is_veg

<xarray.DataArray (y: 12510, x: 10580, band: 4)>
dask.array<mul, shape=(12510, 10580, 4), dtype=uint8, chunksize=(3127, 2645, 4), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) int64 1 2 3 4
Dimensions without coordinates: y, x

In [ ]:
ndvi_mask

In [7]:
pixels = is_veg.stack(z=("x", "y")).drop_vars('z').T
pixels

<xarray.DataArray (z: 132355800, band: 4)>
dask.array<transpose, shape=(132355800, 4), dtype=uint8, chunksize=(6617790, 4), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) int64 1 2 3 4
Dimensions without coordinates: z

In [9]:
pixels = pixels.where(pixels!=0, drop=True)
pixels

<xarray.DataArray (z: 89359635, band: 4)>
dask.array<where, shape=(89359635, 4), dtype=float64, chunksize=(5992439, 4), chunktype=numpy.ndarray>
Coordinates:
  * band     (band) int64 1 2 3 4
Dimensions without coordinates: z

In [10]:
# https://docs.dask.org/en/stable/generated/dask.dataframe.from_array.html#dask.dataframe.from_array
spec = daf.from_array(pixels, chunksize=int(pixels.shape[0]/16))
spec

,0,1,2,3
npartitions=17,,,,
0,float64,float64,float64,float64
5584977,...,...,...,...
...,...,...,...,...
89359632,...,...,...,...
89359634,...,...,...,...


In [11]:
spec['ndvi'] = (spec[3] - spec[0])/(spec[3] + spec[0])
date = item.datetime
kwargs = {'year' : date.year,
         'month' : date.month,
         'day_in_year' : sr.day_in_year(date.day, date.month, date.year)}

spec = spec.assign(**kwargs)
spec

,0,1,2,3,ndvi,year,month,day_in_year
npartitions=17,,,,,,,,
0,float64,float64,float64,float64,float64,int64,int64,int64
5584977,...,...,...,...,...,...,...,...
...,...,...,...,...,...,...,...,...
89359632,...,...,...,...,...,...,...,...
89359634,...,...,...,...,...,...,...,...


In [12]:
# open pre-trained random forest classifier
rfc = load('spectral_rfc.joblib') 
print('loaded model')

loaded model


In [13]:
da_pixels = spec.to_dask_array()
da_pixels

dask.array<values, shape=(nan, 8), dtype=float64, chunksize=(nan, 8), chunktype=numpy.ndarray>

In [14]:
scene_preds = rfc.predict(da_pixels)
scene_preds

dask.array<_predict, shape=(nan,), dtype=int64, chunksize=(nan,), chunktype=numpy.ndarray>

In [15]:
t0 = time.time()
preds = scene_preds.compute()
print('finished predicting\n   time taken to predict: ', time.time() - t0,' s')

finished predicting
   time taken to predict:  242.54775762557983  s


In [16]:
preds

array([0, 0, 0, ..., 0, 0, 0])

distributed.client - ERROR - Failed to reconnect to scheduler after 30.00 seconds, closing client


In [ ]:
# ---------------------------------------
# recover pixel indices for iceplant classifications
preds_df = pd.DataFrame(preds, 
                     columns=['is_iceplant'], 
                     index = is_veg.index)
is_iceplant_index = preds_df[preds_df.is_iceplant == 1].index.to_numpy()
non_iceplant_index = preds_df[preds_df.is_iceplant == 0].index.to_numpy()

# ---------------------------------------
# reconstruct indices into image
indices = [non_iceplant_index,
           is_iceplant_index, 
           not_veg_index,
           water_index]
values = [0,    # values assigned to pixels from each index
          1,
          2,
          3]
t0 = time.time()
reconstruct = rm.indices_to_image(raster.shape[1], raster.shape[2], indices, values, back_value=100)
print('reconstructed image\n   time taken to reconstruct: ', time.time() - t0,' s')

# ---------------------------------------
# save raster 
with rasterio.open(
    os.path.join(os.getcwd(),'temp',filename),  # file path
    'w',           # w = write
    driver = 'GTiff', # format
    height = reconstruct.shape[0], 
    width = reconstruct.shape[1],
    count = 1,  # number of raster bands in the dataset
    dtype = rasterio.uint8,
    crs = raster.rio.crs,
    transform = raster.rio.transform(),
) as dst:
    dst.write(reconstruct.astype(rasterio.uint8), 1)
# ---------------------------------------
print('total tme:', time.time() - t_alpha)